#PRACTICA 04 PARTE 2 MINERIA DE DATOS
---
```
Docente    : Carlos Fernando Montoya Cubas
Alumno     : Cesar Rodrigo Ttito Quilca
Codigo     : 140998
Proposito  : Realizar MapReduce en el Bag of Words(BoW) en Pyspark y el TF-IDF
```
---



In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=448b5d6d9debce78cb0d854bb42290026f67a2ccd64d21f187deca34c7e44a14
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()


In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### BAG OF WORDS (Bow)

Es una técnica de procesamiento de lenguaje natural de modelado de texto, es aqul texto que es desordenado y no estructurado, y los algoritmos de aprendizaje automático prefieren entradas de longitud fija estructuradas y bien definidas y, al utilizar la técnica Bag-of-Words, podemos convertir textos de longitud variable en una longitud fija. `vector.`





In [4]:
import math
#creacion para el tokenizer o bag of words
data=[(1,'La familia Flores tiene un florero lleno de flores Viven en la calle Florida del distrito de Flores, pasan los días regando y viendo florecer las flores de su florería que es la mejor de Florida'),
      (2,"Mi mama me mima y yo mimo a mi mama"),
      (3,"Tres tristes tigres tragaban trigo en un trigal en tres tristes trastos tragaban trigo tres tristes tigres")]
lines=sc.parallelize(data)
lines.collect()
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
map1.collect()
#utilizacion del map reduce
reduce=map1.reduceByKey(lambda x,y:x+y)
#imprimir el baw of word de las 3 oraciones
reduce.collect()


[((1, 'La'), 1),
 ((1, 'tiene'), 1),
 ((1, 'un'), 1),
 ((1, 'lleno'), 1),
 ((1, 'de'), 4),
 ((1, 'la'), 2),
 ((1, 'Flores,'), 1),
 ((1, 'días'), 1),
 ((1, 'regando'), 1),
 ((1, 'las'), 1),
 ((1, 'es'), 1),
 ((2, 'Mi'), 1),
 ((2, 'mama'), 2),
 ((2, 'mima'), 1),
 ((2, 'y'), 1),
 ((2, 'yo'), 1),
 ((2, 'mimo'), 1),
 ((2, 'mi'), 1),
 ((3, 'tragaban'), 2),
 ((3, 'un'), 1),
 ((1, 'familia'), 1),
 ((1, 'Flores'), 1),
 ((1, 'florero'), 1),
 ((1, 'flores'), 2),
 ((1, 'Viven'), 1),
 ((1, 'en'), 1),
 ((1, 'calle'), 1),
 ((1, 'Florida'), 2),
 ((1, 'del'), 1),
 ((1, 'distrito'), 1),
 ((1, 'pasan'), 1),
 ((1, 'los'), 1),
 ((1, 'y'), 1),
 ((1, 'viendo'), 1),
 ((1, 'florecer'), 1),
 ((1, 'su'), 1),
 ((1, 'florería'), 1),
 ((1, 'que'), 1),
 ((1, 'mejor'), 1),
 ((2, 'me'), 1),
 ((2, 'a'), 1),
 ((3, 'Tres'), 1),
 ((3, 'tristes'), 3),
 ((3, 'tigres'), 2),
 ((3, 'trigo'), 2),
 ((3, 'en'), 2),
 ((3, 'trigal'), 1),
 ((3, 'tres'), 2),
 ((3, 'trastos'), 1)]

###Calculo TF- IDF

In [5]:
#Calculamos TF
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()
#Calculamos IDF
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()
rdd=tf.join(idf)
rdd.collect()
#calculamos TF-IDF
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()


In [6]:
#Mostrar Resultado
park = SparkSession(sc)
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2]))
hasattr(rdd, "toDF")
rdd.toDF(["ID","Token","Bag_of_Word","TF-IDF"]).show(50)

+---+--------+-----------+-------------------+
| ID|   Token|Bag_of_Word|             TF-IDF|
+---+--------+-----------+-------------------+
|  1|       y|          1|0.17609125905568124|
|  1| florero|          1|0.47712125471966244|
|  1|  flores|          2|0.47712125471966244|
|  1|   pasan|          1|0.47712125471966244|
|  1|     los|          1|0.47712125471966244|
|  1|  viendo|          1|0.47712125471966244|
|  1|florecer|          1|0.47712125471966244|
|  1|   mejor|          1|0.47712125471966244|
|  1|      La|          1|0.47712125471966244|
|  1|   tiene|          1|0.47712125471966244|
|  1|   lleno|          1|0.47712125471966244|
|  1|      la|          2|0.47712125471966244|
|  1|    días|          1|0.47712125471966244|
|  1| regando|          1|0.47712125471966244|
|  1|     las|          1|0.47712125471966244|
|  1|      es|          1|0.47712125471966244|
|  1| familia|          1|0.47712125471966244|
|  1|  Flores|          1|0.47712125471966244|
|  1|   Viven